###### NAME: JANMEJAY MOHANTY <br> COURSE: BIA 660 WS <br> COURSE NAME: WEB MINING <br> CWID: 20009315 <br> EMAIL: JMOHANTY@STEVENS.EDU <br> PROGRAM: TEXT MINING

In [94]:
# Importing natural language toolkit and csv libraries
import csv
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize

In [95]:
# Function that loads a lexicon of positive words to a set and returns the set
def loadLexicon(fname):
    newLex=set()
    lex_conn=open(fname)
    
    # Add every word in the file to the set
    for line in lex_conn:
        newLex.add(line.strip()) # Remember to strip to remove the lin-change character
    lex_conn.close()

    return newLex

In [96]:
# Function for opinion that a review expresses on a noun
def noun_expression(review_text):

    # Load the positive and negative lexicons into sets
    posLex=loadLexicon('positive-words.txt')
    negLex=loadLexicon('negative-words.txt')

    sentence = sent_tokenize(review_text) # Split the review into sentences

    noun_sentiment = {} # Maps each noun to the number of times it appears in the same sentence as a positive or negative word

    for i in range(len(sentence)):

        pos = 0  # Postive words counts
        neg = 0  # Negative words counts

        nouns_in_sentence=set() # Set of all the nouns in the sentence

        words = word_tokenize(sentence[i])  # Split the review into words

        tagged_words = nltk.pos_tag(words)  # POS tagging for the words in the sentence

        for tagged_word in tagged_words:

            if tagged_word[1].startswith('NN'): # If the word is a noun
                noun = tagged_word[0].lower()   # Lower case the noun
                if len(noun)<3:continue         # Ignore nouns with less than 3 characters
                nouns_in_sentence.add(noun)     # Add the noun to the set
        
            if tagged_word[1].startswith('JJ') and (tagged_word[0] in posLex):
                pos+=1  # Positive word count
            if tagged_word[1].startswith('JJ') and (tagged_word[0] in negLex):
                neg+=1  # Negative word count
            
            for noun in nouns_in_sentence: # For each noun that we found in the sentence
                noun_sentiment[noun]=noun_sentiment.get(noun,[(0,0)])+[(pos,neg)]

        noun_dict = {} 
        for k,v in noun_sentiment.items():
            res = [sum(tup) for tup in zip(*v)]
            noun_dict[k]=res    

        for k,v in noun_dict.items():
            if(v[0]==v[1]):                  # P = v[0] and N = v[1]
                noun_dict[k]='neutral'       # The review on the noun is neutral (P==N)
            elif(v[0]>v[1]):
                noun_dict[k]='positive'      # The review on the noun is positive (P>N)
            else:
                noun_dict[k]='negative'      # The review on the noun is negative (P<N)

    return noun_dict                

In [97]:
def parse(file,idx1,idx2):
    
    fin = open(file,encoding='utf8')

    reader = csv.reader(fin)

    reader = list(reader)
    
    review = []

    review.append(reader[idx1-1])
    review.append(reader[idx2-1])

    #print(len(review))
    #print("Review {}:".format(idx1),review[0][0])
    #print("_________________________________________________________________")
    #print("Review {}:".format(idx2),review[1][0])

    r1 = noun_expression(review[0][0])
    r2 = noun_expression(review[1][0])      

    opp_opinion = []       # Opposite opinion list

    # Comparing positive vs negative opinion of two reviews
    for k1,v1 in r1.items():
        for k2,v2 in r2.items():
            if (k1 == k2):
                if (v1 != v2):
                    opp_opinion.append(k1)
                
    fin.close()

    return opp_opinion


In [98]:
opposite_opinion = parse('amazonreviews.csv',1,2)       # parse('input_file','index of specific review', 'index of specific review')

In [99]:
opposite_opinion.sort()          # Sorts normally in alphabetical order
opposite_opinion.sort(key=len)   # Again sorts in ascending length order

In [100]:
print("Two reviews express an opposite opinion for the attribute:\n",opposite_opinion)

Two reviews express an opposite opinion for the attribute:
 ['app', 'case', 'head', 'sony', 'time', 'noise', 'office', 'buttons', 'quality', 'support', 'headphones', 'sennheiser']
